## Preprocess MAANG companies stock market data

- Import libraries  
- Load raw data (CSV, API, etc.)  
- Handle missing values / duplicates  
- Feature engineering  
- Save processed data to file (e.g., `data/processed.csv`)


In [1]:
import pandas as pd
import numpy as np
import matplotlib as mpl

In [3]:
raw_AAPL = pd.read_csv('data/Apple.csv')
raw_AAPL.head(3)

,Open,High,Low,Close,Adj Close,Volume,Date
0,0.936384,1.004464,0.907924,0.999442,0.850643,535796800,2000-01-03
1,0.966518,0.987723,0.903460,0.915179,0.778926,512377600,2000-01-04
2,0.926339,0.987165,0.919643,0.928571,0.790324,778321600,2000-01-05
